## SQLAlchemy Queries

In this section, we will cover all major aspects of using SQLAlchemy to query required information from the database. In SQLAlchemy we use `query` method to achieve it as shown below.

```python
query = session.query(MappedClass)
```
Lets take a previous example and will use it to show the power of `query` method.

In [159]:
# Common code starts(1)
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship


Base = declarative_base()
# Common code ends (1)


class Students(Base):
    __tablename__ = "students"
    id = Column(Integer, primary_key=True)
    name = Column(String)
    school_id = Column(Integer, ForeignKey('school.id'))
    # school = relationship("School", backref="students")


class School(Base):
    __tablename__ = 'school'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    students = relationship("Students", backref="school",
                            cascade="all,delete,delete-orphan")

    
DB_FILE = os.path.join("db", "many2one_3_3.sqlite3")

try:
    os.remove(DB_FILE)
except Exception as e:
    print(e)

# echo=False for prod code
engine = create_engine('sqlite:///' + DB_FILE, echo=True)
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()
dms = School()
dms.name = "Demonstration Multipurpose Higher Secondary School"
session.add(dms)

session.commit()

students = [
    "Sachin Shah",
    "Satendra",
    "Rajeev Chaturvedi"]

session.add_all([Students(name=student, school=dms) for student in students])
cdac = School(name="CDAC, Hyderabad")
session.add(cdac)
session.flush()
session.commit()

# --------------------------------------------------
cdac_students = [
    "Manish Gupta",
    "Viral Kamdar",
    "Pinakin Purohit",
    "Nitin Srivastava"]

session.add_all([Students(name=student, school=cdac)
                 for student in cdac_students])
session.flush()
session.commit()

2019-05-21 07:43:01,783 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-21 07:43:01,793 INFO sqlalchemy.engine.base.Engine ()
2019-05-21 07:43:01,795 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-21 07:43:01,796 INFO sqlalchemy.engine.base.Engine ()
2019-05-21 07:43:01,798 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("students")
2019-05-21 07:43:01,800 INFO sqlalchemy.engine.base.Engine ()
2019-05-21 07:43:01,803 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("school")
2019-05-21 07:43:01,805 INFO sqlalchemy.engine.base.Engine ()
2019-05-21 07:43:01,808 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE school (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (id)
)


2019-05-21 07:43:01,809 INFO sqlalchemy.engine.base.Engine ()
2019-05-21 07:43:01,813 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-21 07:43:01,815 INFO sqlalchemy.engine.base.Engine 
CREATE 

The below query will return all the students present in the database

In [146]:
for student in session.query(Students):
    print(student)

In [148]:
# Lets get the attributes of student object
for student in session.query(Students):
    print(student.name)

Sachin Shah
Satendra
Rajeev Chaturvedi
Manish Gupta
Viral Kamdar
Pinakin Purohit
Nitin Srivastava


If we wish to only get user name then we can use the following query

In [155]:
# The result set will only have name 
# ('Sachin Shah',) is not a tuple but resutlset object
for student in session.query(Students.name):
    print(student, student.name)

('Sachin Shah',) Sachin Shah
('Satendra',) Satendra
('Rajeev Chaturvedi',) Rajeev Chaturvedi
('Manish Gupta',) Manish Gupta
('Viral Kamdar',) Viral Kamdar
('Pinakin Purohit',) Pinakin Purohit
('Nitin Srivastava',) Nitin Srivastava


In [156]:
for student in session.query(Students):
    print(student.name, "is studying in", student.school.name)

Sachin Shah is studying in Demonstration Multipurpose Higher Secondary School
Satendra is studying in Demonstration Multipurpose Higher Secondary School
Rajeev Chaturvedi is studying in Demonstration Multipurpose Higher Secondary School
Manish Gupta is studying in CDAC, Hyderabad
Viral Kamdar is studying in CDAC, Hyderabad
Pinakin Purohit is studying in CDAC, Hyderabad
Nitin Srivastava is studying in CDAC, Hyderabad


Say we wants to rename the `name` of student in result to `StudentName` than we can use `label` method as shown

In [170]:
for student in session.query(Students.name.label("StudentName")):
    print(student.StudentName)

2019-05-21 07:47:35,716 INFO sqlalchemy.engine.base.Engine SELECT students.name AS "StudentName" 
FROM students
2019-05-21 07:47:35,717 INFO sqlalchemy.engine.base.Engine ()
Sachin Shah
Satendra
Rajeev Chaturvedi
Manish Gupta
Viral Kamdar
Pinakin Purohit
Nitin Srivastava


In [171]:
# This will also work
name = Students.name.label("StudentName")

for student in session.query(name):
    print(student.StudentName)

2019-05-21 07:47:57,286 INFO sqlalchemy.engine.base.Engine SELECT students.name AS "StudentName" 
FROM students
2019-05-21 07:47:57,291 INFO sqlalchemy.engine.base.Engine ()
Sachin Shah
Satendra
Rajeev Chaturvedi
Manish Gupta
Viral Kamdar
Pinakin Purohit
Nitin Srivastava


In [165]:
# $$TODO$$
# for student in session.query(Students.name.label("Student Name")):
#    print(student.get("Student Name"))

We can provide similar name to entire entity also using `aliased` method as shown in below example

In [169]:
from sqlalchemy.orm import aliased
a_students = aliased(Students, name='StudentObject')

for row in session.query(a_students, a_students.name).all():
    print(row.StudentObject.name, "\t=", row.StudentObject)

2019-05-21 07:45:22,921 INFO sqlalchemy.engine.base.Engine SELECT "StudentObject".id AS "StudentObject_id", "StudentObject".name AS "StudentObject_name", "StudentObject".school_id AS "StudentObject_school_id" 
FROM students AS "StudentObject"
2019-05-21 07:45:22,924 INFO sqlalchemy.engine.base.Engine ()
Sachin Shah 	= <__main__.Students object at 0x111d2f908>
Satendra 	= <__main__.Students object at 0x111d38d68>
Rajeev Chaturvedi 	= <__main__.Students object at 0x111d38dd8>
Manish Gupta 	= <__main__.Students object at 0x111d38cf8>
Viral Kamdar 	= <__main__.Students object at 0x111d38cc0>
Pinakin Purohit 	= <__main__.Students object at 0x111d38908>
Nitin Srivastava 	= <__main__.Students object at 0x111d38f60>


We can also use slicing on the query.

In [113]:
for student in session.query(Students)[:2]:
    print(student.name, "is studying in", student.school.name)

2019-05-21 05:57:47,282 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students
 LIMIT ? OFFSET ?
2019-05-21 05:57:47,283 INFO sqlalchemy.engine.base.Engine (2, 0)
Sachin Shah is studying in Demonstration Multipurpose Higher Secondary School
Satendra is studying in Demonstration Multipurpose Higher Secondary School


### Ordering the query.

we can use `order_by` to achieve order in the query results as shown in the below example.

In [173]:
for student in session.query(Students.name).order_by(Students.name):
    print(student.name)

2019-05-21 07:49:45,225 INFO sqlalchemy.engine.base.Engine SELECT students.name AS students_name 
FROM students ORDER BY students.name
2019-05-21 07:49:45,228 INFO sqlalchemy.engine.base.Engine ()
Manish Gupta
Nitin Srivastava
Pinakin Purohit
Rajeev Chaturvedi
Sachin Shah
Satendra
Viral Kamdar


In [178]:
for student in session.query(Students.name).order_by(Students.school_id):
    print(student.name)

2019-05-21 07:53:19,009 INFO sqlalchemy.engine.base.Engine SELECT students.name AS students_name 
FROM students ORDER BY students.school_id
2019-05-21 07:53:19,012 INFO sqlalchemy.engine.base.Engine ()
Sachin Shah
Satendra
Rajeev Chaturvedi
Manish Gupta
Viral Kamdar
Pinakin Purohit
Nitin Srivastava


##### Cascaded `order_by`

In [180]:
for student in session.query(Students.name).order_by(Students.school_id).order_by(Students.name):
    print(student.name)

2019-05-21 07:53:32,960 INFO sqlalchemy.engine.base.Engine SELECT students.name AS students_name 
FROM students ORDER BY students.school_id, students.name
2019-05-21 07:53:32,962 INFO sqlalchemy.engine.base.Engine ()
Rajeev Chaturvedi
Sachin Shah
Satendra
Manish Gupta
Nitin Srivastava
Pinakin Purohit
Viral Kamdar


### Filtering the Query

We can use `filter_by` method to apply filtering in the query

In [181]:
cdac = session.query(School).filter_by(name="CDAC, Hyderabad").first()
print(cdac.name)

2019-05-21 07:54:05,936 INFO sqlalchemy.engine.base.Engine SELECT school.id AS school_id, school.name AS school_name 
FROM school 
WHERE school.name = ?
 LIMIT ? OFFSET ?
2019-05-21 07:54:05,943 INFO sqlalchemy.engine.base.Engine ('CDAC, Hyderabad', 1, 0)
CDAC, Hyderabad


#### Common Filter Operators

##### equals:

In [182]:
cdac = session.query(School).filter(School.name == "CDAC, Hyderabad").first()
print(cdac.name)

2019-05-21 07:55:15,743 INFO sqlalchemy.engine.base.Engine SELECT school.id AS school_id, school.name AS school_name 
FROM school 
WHERE school.name = ?
 LIMIT ? OFFSET ?
2019-05-21 07:55:15,746 INFO sqlalchemy.engine.base.Engine ('CDAC, Hyderabad', 1, 0)
CDAC, Hyderabad


In [184]:
for student in session.query(Students).filter(Students.school_id == 1):
    print(student.name)

2019-05-21 07:55:37,007 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.school_id = ?
2019-05-21 07:55:37,009 INFO sqlalchemy.engine.base.Engine (1,)
Sachin Shah
Satendra
Rajeev Chaturvedi


In [186]:
for student in session.query(Students).filter(Students.name=="Satendra"):
    print(student.name)

2019-05-21 07:56:40,800 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name = ?
2019-05-21 07:56:40,803 INFO sqlalchemy.engine.base.Engine ('Satendra',)
Satendra


##### not equals:

In [187]:
for student in session.query(Students).filter(Students.name!="Satendra"):
    print(student.name)

2019-05-21 07:57:04,036 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name != ?
2019-05-21 07:57:04,040 INFO sqlalchemy.engine.base.Engine ('Satendra',)
Sachin Shah
Rajeev Chaturvedi
Manish Gupta
Viral Kamdar
Pinakin Purohit
Nitin Srivastava


LIKE:

In [190]:
for student in session.query(Students).filter(Students.name.like("%S%")):
    print(student.name)

2019-05-21 07:58:29,719 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name LIKE ?
2019-05-21 07:58:29,722 INFO sqlalchemy.engine.base.Engine ('%S%',)
Sachin Shah
Satendra
Manish Gupta
Nitin Srivastava


In [194]:
for student in session.query(Students).filter(Students.name.like("S%")):
    print(student.name)

2019-05-21 08:00:05,370 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name LIKE ?
2019-05-21 08:00:05,372 INFO sqlalchemy.engine.base.Engine ('S%',)
Sachin Shah
Satendra


ILIKE (case-insensitive LIKE):

In [195]:
for student in session.query(Students).filter(Students.name.ilike("%S%")):
    print(student.name)

2019-05-21 08:00:19,772 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE lower(students.name) LIKE lower(?)
2019-05-21 08:00:19,774 INFO sqlalchemy.engine.base.Engine ('%S%',)
Sachin Shah
Satendra
Manish Gupta
Nitin Srivastava


##### IN (`in_`):

In [204]:
for student in session.query(Students).filter(Students.name.in_(['Sachin Shah', 'Manish Gupta'])):
    print(student, student.name)

2019-05-21 08:04:01,630 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name IN (?, ?)
2019-05-21 08:04:01,632 INFO sqlalchemy.engine.base.Engine ('Sachin Shah', 'Manish Gupta')
<__main__.Students object at 0x111a2ca20> Sachin Shah
<__main__.Students object at 0x111a2e8d0> Manish Gupta


##### NOT IN (`~`): 

In [205]:
for student in session.query(Students).filter(~Students.name.in_(['Sachin Shah', 'Manish Gupta'])):
    print(student, student.name)

2019-05-21 08:04:26,830 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name NOT IN (?, ?)
2019-05-21 08:04:26,833 INFO sqlalchemy.engine.base.Engine ('Sachin Shah', 'Manish Gupta')
<__main__.Students object at 0x111a2c198> Satendra
<__main__.Students object at 0x111a2c2b0> Rajeev Chaturvedi
<__main__.Students object at 0x111a2c550> Viral Kamdar
<__main__.Students object at 0x111a2c5c0> Pinakin Purohit
<__main__.Students object at 0x111d38f60> Nitin Srivastava


##### IS NULL:

In [206]:
for student in session.query(Students).filter(Students.name == None):
    print(student, student.name)

2019-05-21 08:05:50,804 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name IS NULL
2019-05-21 08:05:50,805 INFO sqlalchemy.engine.base.Engine ()


##### NOT NULL

In [210]:
for student in session.query(Students).filter(Students.name != None):
    print(student, student.name, student.school.id)

2019-05-21 08:08:18,239 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name IS NOT NULL
2019-05-21 08:08:18,242 INFO sqlalchemy.engine.base.Engine ()
<__main__.Students object at 0x111d3a518> Sachin Shah 1
<__main__.Students object at 0x111d3add8> Satendra 1
<__main__.Students object at 0x111d3ae80> Rajeev Chaturvedi 1
<__main__.Students object at 0x111a2e0f0> Manish Gupta 2
<__main__.Students object at 0x111a2ee80> Viral Kamdar 2
<__main__.Students object at 0x111a2ef28> Pinakin Purohit 2
<__main__.Students object at 0x111d38f60> Nitin Srivastava 2


##### AND (`and_`):

In [217]:
from sqlalchemy import and_

for student in session.query(Students).filter(and_(Students.name.like("S%"), Students.school_id==1)):
    print(student, student.name)

2019-05-21 08:10:59,512 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name LIKE ? AND students.school_id = ?
2019-05-21 08:10:59,514 INFO sqlalchemy.engine.base.Engine ('S%', 1)
<__main__.Students object at 0x111a2ceb8> Sachin Shah
<__main__.Students object at 0x111a2c198> Satendra


##### OR (or_):

In [220]:
from sqlalchemy import or_

for student in session.query(Students).filter(or_(Students.name.like("%s%"), Students.school_id==1)):
    print(student, student.name)

2019-05-21 08:12:00,820 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name LIKE ? OR students.school_id = ?
2019-05-21 08:12:00,822 INFO sqlalchemy.engine.base.Engine ('%s%', 1)
<__main__.Students object at 0x111a2eb00> Sachin Shah
<__main__.Students object at 0x111a2e048> Satendra
<__main__.Students object at 0x111a2eeb8> Rajeev Chaturvedi
<__main__.Students object at 0x111a2eb70> Manish Gupta
<__main__.Students object at 0x111d38f60> Nitin Srivastava


##### MATCH:

In [225]:
## TODO
for student in session.query(Students).filter(Students.name.match('Sachin%')):
    print(student)

2019-05-21 08:16:19,831 INFO sqlalchemy.engine.base.Engine SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name MATCH ?
2019-05-21 08:16:19,835 INFO sqlalchemy.engine.base.Engine ('Sachin%',)


OperationalError: (sqlite3.OperationalError) unable to use function MATCH in the requested context
[SQL: SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name MATCH ?]
[parameters: ('Sachin%',)]
(Background on this error at: http://sqlalche.me/e/e3q8)

#### `count`

In [229]:
nos = session.query(Students).filter(Students.name.like("%S%")).count()
print(nos)

2019-05-21 08:20:31,134 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name LIKE ?) AS anon_1
2019-05-21 08:20:31,136 INFO sqlalchemy.engine.base.Engine ('%S%',)
4


In [231]:
from sqlalchemy import and_

nos = session.query(Students).filter(and_(Students.name.like("S%"), Students.school_id==1)).count()
print(nos)

2019-05-21 08:21:10,489 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name LIKE ? AND students.school_id = ?) AS anon_1
2019-05-21 08:21:10,491 INFO sqlalchemy.engine.base.Engine ('S%', 1)
2


In [233]:
# shotcut for query
student_query = session.query(Students)
nos = student_query.filter(and_(Students.name.like("S%"), Students.school_id==1)).count()
print(nos)

2019-05-21 08:21:42,448 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT students.id AS students_id, students.name AS students_name, students.school_id AS students_school_id 
FROM students 
WHERE students.name LIKE ? AND students.school_id = ?) AS anon_1
2019-05-21 08:21:42,450 INFO sqlalchemy.engine.base.Engine ('S%', 1)
2
